# Comparación general

Este notebook implementa 6 enfoques distintos para resolver el clásico problema del granjero que debe cruzar un río con un lobo, una cabra y una col.

Cada enfoque representa un método de exploración del espacio de estados:

1. Representación del estado
2. BFS (cola)
3. DFS (pila)
4. Backtracking recursivo
5. Grafo explícito
6. Árbol de búsqueda (nodos con padre)

Al final se presentan conclusiones comparativas sobre desempeño, eficiencia y calidad de solución.

In [1]:
# Representación del estado y funciones base

from collections import deque
from itertools import product

# Verifica si un estado es seguro
def is_safe(state):
    G, L, C, Co = state
    # Lobo y cabra solos
    if L == C and G != L:
        return False
    # Cabra y col solos
    if C == Co and G != C:
        return False
    return True

def flip(x):
    return "R" if x == "L" else "L"

# Genera próximos estados válidos
def neighbors(state):
    G, L, C, Co = state
    moves = []
    new_G = flip(G)

    # Granjero solo
    s = (new_G, L, C, Co)
    if is_safe(s): moves.append(s)

    # Granjero + lobo
    if L == G:
        s = (new_G, flip(L), C, Co)
        if is_safe(s): moves.append(s)

    # Granjero + cabra
    if C == G:
        s = (new_G, L, flip(C), Co)
        if is_safe(s): moves.append(s)

    # Granjero + col
    if Co == G:
        s = (new_G, L, C, flip(Co))
        if is_safe(s): moves.append(s)

    return moves

start = ("L", "L", "L", "L")
goal  = ("R", "R", "R", "R")


In [ ]:
# BFS — Solución más corta
def solve_bfs():
    queue = deque([(start, [start])])
    visited = set([start])
    
    while queue:
        state, path = queue.popleft()
        
        if state == goal:
            return path
        
        for nxt in neighbors(state):
            if nxt not in visited:
                visited.add(nxt)
                queue.append((nxt, path + [nxt]))

path_bfs = solve_bfs()
path_bfs


[('L', 'L', 'L', 'L'),
 ('R', 'L', 'R', 'L'),
 ('L', 'L', 'R', 'L'),
 ('R', 'R', 'R', 'L'),
 ('L', 'R', 'L', 'L'),
 ('R', 'R', 'L', 'R'),
 ('L', 'R', 'L', 'R'),
 ('R', 'R', 'R', 'R')]

In [3]:
# DFS — Solución no necesariamente mínima
def solve_dfs():
    stack = [(start, [start])]
    visited = set([start])
    
    while stack:
        state, path = stack.pop()
        
        if state == goal:
            return path
        
        for nxt in neighbors(state):
            if nxt not in visited:
                visited.add(nxt)
                stack.append((nxt, path + [nxt]))

path_dfs = solve_dfs()
path_dfs


[('L', 'L', 'L', 'L'),
 ('R', 'L', 'R', 'L'),
 ('L', 'L', 'R', 'L'),
 ('R', 'L', 'R', 'R'),
 ('L', 'L', 'L', 'R'),
 ('R', 'R', 'L', 'R'),
 ('L', 'R', 'L', 'R'),
 ('R', 'R', 'R', 'R')]

In [4]:
# Backtracking — Exploración exhaustiva recursiva

def solve_backtracking():
    visited = set()
    path = []

    def backtrack(state):
        if state == goal:
            path.append(state)
            return True
        
        visited.add(state)
        path.append(state)
        
        for nxt in neighbors(state):
            if nxt not in visited:
                if backtrack(nxt):
                    return True
        
        path.pop()
        return False

    backtrack(start)
    return path

path_bt = solve_backtracking()
path_bt


[('L', 'L', 'L', 'L'),
 ('R', 'L', 'R', 'L'),
 ('L', 'L', 'R', 'L'),
 ('R', 'R', 'R', 'L'),
 ('L', 'R', 'L', 'L'),
 ('R', 'R', 'L', 'R'),
 ('L', 'R', 'L', 'R'),
 ('R', 'R', 'R', 'R')]

In [5]:
# Grafo explícito — Generación completa del estado-espacio
def build_graph():
    graph = {}
    for state in product(["L", "R"], repeat=4):
        if is_safe(state):
            graph[state] = neighbors(state)
    return graph

graph = build_graph()
len(graph)


#BFS sobre el grafo:
def bfs_graph(graph):
    queue = deque([(start, [start])])
    visited = set([start])

    while queue:
        state, path = queue.popleft()
        
        if state == goal:
            return path
        
        for nxt in graph[state]:
            if nxt not in visited:
                visited.add(nxt)
                queue.append((nxt, path + [nxt]))

path_graph = bfs_graph(graph)
path_graph


[('L', 'L', 'L', 'L'),
 ('R', 'L', 'R', 'L'),
 ('L', 'L', 'R', 'L'),
 ('R', 'R', 'R', 'L'),
 ('L', 'R', 'L', 'L'),
 ('R', 'R', 'L', 'R'),
 ('L', 'R', 'L', 'R'),
 ('R', 'R', 'R', 'R')]

In [6]:
# Árbol de búsqueda — Nodos enlazados con padre
class Node:
    def __init__(self, state, parent=None):
        self.state = state
        self.parent = parent

def solve_tree():
    root = Node(start)
    queue = [root]
    visited = set([start])
    
    while queue:
        node = queue.pop(0)
        
        if node.state == goal:
            # reconstruir camino
            path = []
            while node:
                path.append(node.state)
                node = node.parent
            return list(reversed(path))
        
        for nxt in neighbors(node.state):
            if nxt not in visited:
                visited.add(nxt)
                queue.append(Node(nxt, node))

path_tree = solve_tree()
path_tree


[('L', 'L', 'L', 'L'),
 ('R', 'L', 'R', 'L'),
 ('L', 'L', 'R', 'L'),
 ('R', 'R', 'R', 'L'),
 ('L', 'R', 'L', 'L'),
 ('R', 'R', 'L', 'R'),
 ('L', 'R', 'L', 'R'),
 ('R', 'R', 'R', 'R')]

In [7]:
# Comparación de resultados
results = {
    "BFS": path_bfs,
    "DFS": path_dfs,
    "Backtracking": path_bt,
    "Grafo + BFS": path_graph,
    "Árbol de búsqueda": path_tree,
}

for method, path in results.items():
    print(f"{method}: {len(path)} pasos")


BFS: 8 pasos
DFS: 8 pasos
Backtracking: 8 pasos
Grafo + BFS: 8 pasos
Árbol de búsqueda: 8 pasos


# Conclusiones comparativas
## 1. BFS (cola)
* Ganador absoluto.
* Encuentra SIEMPRE la ruta más corta: 7 movimientos.
* Eficiente, simple y evita ciclos.
* Ideal para problemas pequeños y sin pesos.
## 2. DFS (pila)
* Encuentra alguna solución, pero NO garantiza que sea la más corta.
* Dependiendo del orden de vecinos, puede generar rutas más largas.
* Puede atascarse explorando profundamente antes de probar soluciones mejores.
## 3. Backtracking
* Encuentra solución garantizada.
* Consume más tiempo porque explora todos los caminos posibles.
* No es eficiente, pero es didáctico.
## 4. Grafo explícito
* Permite estudiar el espacio de estados completo.
* En este problema hay 16 estados posibles, 10 seguros.
* Útil si quieres analizar conectividad y estructura del problema.
## 5. Árbol de búsqueda
* Es básicamente BFS con nodos enlazados.
* Facilita reconstruir rutas.
* Desempeño similar a BFS, pero con más memoria por usar objetos.